In [1]:
# 필요한 모듈 import
from __future__ import division
import io
import os
import sys
import time
import numpy as np
import tensorflow as tf
import cv2
from keras.models import load_model
from tensorflow.keras.utils import img_to_array
import imutils
import numpy as np
import dlib 
import datetime
import math 
import re
from google.cloud import speech
import pyaudio
from six.moves import queue
import random
from multiprocessing import Process, Value
import multiprocessing as mp
from videoRecogntion import aaa  # faceDetect 해주는 함수 import
from gptTest import main2, kogpt2  # stt, 키워드 추출, 문장 생성

# GUI
from box import Box  # GUI 카메랄 가져오는 용
from pathlib import Path
from PIL import ImageTk
import PIL.Image
from tkinter import *
import tkinter as tk


import warnings
warnings.filterwarnings('ignore')  # warning 문구 제거 


os.environ['GOOGLE_APPLICATION_CREDENTIALS']=r"/Users/yujeong/Downloads/gradstt-ac34c3b32796.json"  # Google STT API 계정

2022-12-21 19:06:23.848974: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [4]:
# font = cv2.FONT_ITALIC

# # 개체가 포함된 이미지 영역을 가져와 개체의 포즈를 정의하는 점 위치 집합을 출력하기 위한 predictor  --> 눈 인식에 사용
# predictor = dlib.shape_predictor("/Users/yujeong/Desktop/졸프용/shape_predictor_68_face_landmarks.dat")
# tDetector = TensoflowFaceDector(PATH_TO_CKPT)  #  얼굴인식 Detector

# # 표정 인식 모델 경로
# emotion_model_path = '/Users/yujeong/Downloads/_mini_XCEPTION_model_korean_64_2.hdf5'

# EMOTIONS = ["angry", "scared", "happy", "sad", "surprised", "neutral"] # 6가지 표정
# ratio=[0,0,0,0,0,0] # 시각화를 위한 표정 비율
# eye_fix=0 # 눈 흔들림 시각화를 위한 cnt
# max_boxes_to_draw = 5  # 최대 ㅐ5명까지만 인식할 것

# # 발표가 끝난 후 표정 평균을 내기 위함
# people_emo = [[0, 0, 0, 0, 0, 0],
#              [0, 0, 0, 0, 0, 0],
#              [0, 0, 0, 0, 0, 0],
#              [0, 0, 0, 0, 0, 0],
#              [0, 0, 0, 0, 0, 0],
#              [0, 0, 0, 0, 0, 0],
#              [0, 0, 0, 0, 0, 0],
#              [0, 0, 0, 0, 0, 0],
#              [0, 0, 0, 0, 0, 0],
#              [0, 0, 0, 0, 0, 0]]

In [5]:
import re
import sys

from google.cloud import speech

import pyaudio
from six.moves import queue

# 오디오 레코딩 매개변수
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms

f=open("stt.txt",'w')  # 전체 발화 써주는 용


In [ ]:

 # 공유 메모리를 이용한 멀티 프로세싱        

if __name__ == '__main__':    
    
    
    video_pause = Value('i',0)  # 발표 종료 시 
    
            
    """ 화면 구성 """
    
    win=tk.Tk()
    win.title("뚝딱")
    win.geometry("1000x1000")
    win.configure(background='white')

    mainFrm=tk.Frame(win)
    resFrm=tk.Frame(win)

    mainFrm.grid(row=0,column=0,sticky='nsew')
    resFrm.grid(row=0,column=0,sticky='nsew')
    
    # 메인 화면

    mf1=Frame(mainFrm)
    mf1.pack(side="left")
    
    # 키워드 result 보여줄 화면
    mf2_1=Frame(mainFrm,background='white')
    mf2_1.pack()
    
    
    # 꼬리질문 result 보여줄 화면
    mf2_3=Frame(mainFrm,background='white')
    mf2_3.pack()

    
    """ 버튼 함수 """
    
    def openFrame(frame): # open 함수 
        frame.tkraise()

    
    def keyEvent(): # 키워드 추출 가시화 함수 
        f = open('/Users/yujeong/Desktop/keywords.txt', 'r')
        s2 = f.readline()        
        keyText = str(s2)
        keyword_res.config(text=keyText)


    def event():  # 발표 함수 
        strtBtn['text']='발표중'
        main()

    def exitEvent(): # 발표 끝 함수
        video_pause.value = 1   # 공유변수를 1로 수정함 --> 다른 파일에서 공유 후 기능을 멈춤
        
    
    def tailEvent(): # 꼬리질문 가시화 함수 
        f3 = open('/Users/yujeong/Desktop/question.txt', 'r')
        s3 = f3.readline()
        tailText = str(s3)
        tail_res.config(text = tailText)
        
        
        
    # 결과 화면에서 이미지 넘기는 함수들 
    
    def next_img1():
        global img_num1
        img_num1+=1
        if img_num1>=len(img_list):
            img_num1=0
        canvas1.itemconfig(can_img1,image=img_list[img_num1])

    def pre_img1():
        global img_num1
        img_num1-=1
        if img_num1<0:
            img_num1=len(img_list)-1
        canvas1.itemconfig(can_img1,image=img_list[img_num1])

    def next_img2():
        global img_num2
        img_num2+=1
        if img_num2>=len(eye_list):
            img_num2=0
        canvas2.itemconfig(can_img2,image=eye_list[img_num2])

    def pre_img2():
        global img_num2
        img_num2-=1
        if img_num2<0:
            img_num2=len(eye_list)-1
        canvas2.itemconfig(can_img2,image=eye_list[img_num2])    
    
    
    # 웹캠
    video = Box(mf1, height=600,width=700)
    video.show_frames()  # Show the created Box



    # 글자 및 버튼 GUI 구성
    
    keyword=Label(mf2_1, text='추출 keyword') # fg는 글자 색 지정, font로 글자 설정
    keyword.place(x=140, y=10)
    keyword.pack()
    
    keyword_res=Label(mf2_1, width=28,height=10,relief='solid',wraplength=200,text=' ') # wraplength=줄바꿈 단위, relief=테두리 설정
    keyword_res.place(x=100, y=40)
    keyword_res.pack()
    
    kwBtn=Button(mf2_1,text='키워드 추출하기',padx=10,pady=10,command=keyEvent)
    kwBtn.pack()

    tail=Label(mf2_3, text='꼬리질문') # fg는 글자 색 지정, font로 글자 설정
    tail.place(x=140, y=10)
    tail.pack()
    
    tail_res=Label(mf2_3, width=28,height=10,relief='solid',wraplength=200,text=' ') # wraplength=줄바꿈 단위, relief=테두리 설정
    tail_res.place(x=100, y=40)
    tail_res.pack()
    
    tailBtn=Button(mf2_3,text='꼬리질문 생성하기',padx=10,pady=10,command=tailEvent)
    tailBtn.pack()
     
    mf3=Frame(mainFrm)
    mf3.pack()

    strtBtn=Button(mf3,text='발표 시작',padx=10,pady=10,command=event)
    endBtn=Button(mf3,text='발표 종료',padx=10,pady=10,command=exitEvent)
    # 다음페이지로 화면 전환
    btnToRes=Button(mf3,text='결과 보기',padx=10,pady=10,command=lambda:[openFrame(resFrm)])  

    strtBtn.pack(side="left")
    btnToRes.pack(side="right")
    endBtn.pack(side="right")
    
    
    # 결과 화면
    
    # 표정 결과
    path = os.path.join("/Users/yujeong/Desktop/CNN_test/tensorflow-face-detection-master/save_fig/save_emotion/")
    file_list_emotion = os.listdir(path)
    del file_list_emotion[0]
    file_list_emotion.sort()

    img_list = []
    for file in file_list_emotion[:5]:
        file_path = path + file
        img = PIL.Image.open(file_path)
        img = img.resize((int(img.width / 2), int(img.height / 2)))
        img_list.append(ImageTk.PhotoImage(img))

    # 눈 결과
    path = os.path.join("/Users/yujeong/Desktop/CNN_test/tensorflow-face-detection-master/save_fig/save_eye/")
    file_list_eye = os.listdir(path)
    del file_list_eye[0]
    file_list_eye.sort()

    eye_list = []
    for file in file_list_eye[:5]:
        file_path = path + file
        img = PIL.Image.open(file_path)
        img = img.resize((int(img.width / 2), int(img.height / 2)))
        eye_list.append(ImageTk.PhotoImage(img))

    # 발화 속도 결과 
    img = PIL.Image.open('/Users/yujeong/Desktop/CNN_test/tensorflow-face-detection-master/save_fig/save_speed/speed.png')
    img = img.resize((int(img.width / 3), int(img.height / 3)))
    speed_img=ImageTk.PhotoImage(img)


    rf1=Frame(resFrm)
    rf1.pack()

    faceDetect=Label(rf1, text='표정인식 결과') # fg는 글자 색 지정, font로 글자 설정
    faceDetect.place(x=10, y=10)
    faceDetect.pack()

    forward_btn=Button(rf1,text="이전",command=pre_img1)
    forward_btn.place(x=5, y=10)
    forward_btn.pack(side="left")

    backward_btn=Button(rf1,text="다음",command=next_img1)
    backward_btn.place(x=15, y=10)
    backward_btn.pack(side="right")


    canvas1 = Canvas(rf1,width=230,height=230,bg='black')
    canvas1.place(x=20, y=50)
    canvas1.pack()

    img_num1 = 0
    can_img1=canvas1.create_image(120,115,image=img_list[img_num1])

    # 시선 처리 결과
    rf3=Frame(resFrm)
    rf3.pack()

    eye_move=Label(rf3, text='발화속도 결과') # fg는 글자 색 지정, font로 글자 설정
    eye_move.place(x=10, y=10)
    eye_move.pack()

    forward_btn=Button(rf3,text="이전",command=pre_img2)
    forward_btn.place(x=5, y=10)
    forward_btn.pack(side="left")

    backward_btn=Button(rf3,text="다음",command=next_img2)
    backward_btn.place(x=15, y=10)
    backward_btn.pack(side="right")


    canvas2 = Canvas(rf3,width=230,height=180)
    canvas2.place(x=50, y=100)
    canvas2.pack()

    img_num2 = 0
    can_img2=canvas2.create_image(115,100,image=eye_list[img_num2])

    # 발화 속도 결과
    rf2=Frame(resFrm)
    rf2.pack()

    speed=Label(rf2, text='시선처리 결과') # fg는 글자 색 지정, font로 글자 설정
    speed.place(x=10, y=0)
    speed.pack()

    canvas3 = Canvas(rf1,width=320,height=200)
    canvas3.place(x=50, y=30)
    canvas3.pack()

    canvas3.create_image(125,100,image=speed_img)

    # 버튼
    rf4=Frame(resFrm)
    rf4.pack()

    btnToMain=Button(rf4,text="메인으로 돌아가기",padx=10,pady=10,command=lambda:[openFrame(mainFrm)])

    btnToMain.pack()
    
    openFrame(mainFrm)


    
    ###################
    """멀티 프로세싱"""
    
    # 프로세스 생성 후 시작
    
    # 표정인식
    face_func = Process(target=aaa, args=(video_pause, ))
    face_func.start()

    # 키워드 추출, 질문 생성, 발화속도
    nlp_func = Process(target=main2, args=(video_pause, ))
    nlp_func.start()
    
    # GUI 돌리기 용
    loop_func = Process(target=win.mainloop())
    loop_func.start()

    # 메서드는 join() 메서드가 호출된 프로세스가 종료될 때까지 블록
    face_func.join()
    nlp_func.join()
    loop_func.join()
    


2022-12-21 19:06:37.053795: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-21 19:06:37.732156: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
